In [2]:
%load_ext autoreload

import os, sys
import networkx as nx

sys.path.insert(0, "../")

%autoreload 2
from mikg.kgraph import KGraph
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
kg = KGraph()

In [3]:
kg.load_kgraph_base("../data/", go=True, omnipath=True, opentargets=True, reactome=True, STRING=True)

HMS:0:00:11.045307 626,136 annotations READ: ../data/goa_human.gaf 
../data/go-basic.obo: fmt(1.2) rel(2023-01-01) 46,739 Terms


/usr/local/hdd3/mikg/scripts/../mikg/load_utils.py:202: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  opd  = pd.read_csv(omnipathDB, sep="\t")
/usr/local/hdd3/mikg/scripts/../mikg/load_utils.py:370: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  ot_disease = pd.read_csv("../data/opentargets_disease_associations.tsv", sep="\t")
/usr/local/hdd3/mikg/scripts/../mikg/load_utils.py:306: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdf["score"] = subdf[use_evidences].max(axis=1)/1000


In [4]:
kg.print_kg_info()

In [5]:
print(kg.kg)

DiGraph with 116899 nodes and 3428445 edges


In [6]:
kg.save_kgraph("../data/initial_base_graph.out")

In [4]:
kg = KGraph()
kg.load_kgraph("../data/initial_base_graph.out")

In [5]:
exprDF = pd.read_csv("../seq_data/expression_mean_df.tsv", sep="\t")
cmDF = exprDF[exprDF.group == "Cardiomyocyte"]
cmDF

,group,gene,min,percentile_25,median,percentile_75,max,mean,anum,num,sd
22891,Cardiomyocyte,A1BG,0.218642,0.439190,0.607427,0.831411,2.456248,0.709995,351,351,0.395967
22892,Cardiomyocyte,A1BG-AS1,0.214143,0.456117,0.643483,0.936906,3.038826,0.806031,461,461,0.518502
22893,Cardiomyocyte,A1CF,0.208636,0.422485,0.557386,0.773439,3.025672,0.637636,706,706,0.321588
22894,Cardiomyocyte,A2M,0.181245,0.651118,0.944811,1.398837,3.792171,1.075190,32973,32973,0.547520
22895,Cardiomyocyte,A2M-AS1,0.185317,0.412732,0.539294,0.709554,2.699968,0.602207,3160,3160,0.286947
...,...,...,...,...,...,...,...,...,...,...,...
50958,Cardiomyocyte,ZXDC,0.185317,0.610198,0.829333,1.097115,3.521134,0.891553,35191,35191,0.378761
50959,Cardiomyocyte,ZYG11A,0.224826,0.414667,0.540848,0.712375,2.653949,0.637458,303,303,0.367307
50960,Cardiomyocyte,ZYG11B,0.187795,0.600248,0.812252,1.080384,3.494183,0.880513,33025,33025,0.386580
50961,Cardiomyocyte,ZYX,0.193169,0.445595,0.595571,0.835346,3.001737,0.705213,5338,5338,0.395080


In [6]:
kg.add_gene_expression(cmDF)

28072
Found Genes 22754


In [7]:
kg.get_node_data("CCL2")

{'type': 'gene',
 'score': 0,
 'expression': {'mean': 0.7961505602787607,
  'perc_expr': 1.0,
  'median': 0.6903093253438801}}

In [8]:
kg.kg.edges[('NFE2L2', 'CCL2')]

{'type': 'represses',
 'source': 'omnipath',
 'omnipath_evidences': ['NRF2ome:16246346'],
 'omnipath_type': 'post_translational'}

In [9]:
for edge in kg.get_node_edges("CCL2"):
    
    if not "type" in kg.kg.nodes[edge[1]]:
        print(edge, edge[1],kg.kg.nodes[edge[1]])
        break

In [10]:
def scoring_represses(x, y):
    return x - y

def scoring_activates(x, y):
    return x*y

def scoring_interacts(x,y):
    return x*y

def scoring_null(x,y):
    return 0

scoring_gene_gene_expression = {"represses": scoring_represses, "activates": scoring_activates, "interacts": scoring_interacts, "-": scoring_null}

In [11]:
for edge in kg.get_node_edges("CCL2", in_types=["gene"], out_types=["gene"]):
    
    inExpr = kg.get_node_data(edge[0]).get("expression", {}).get("mean", 0)
    outExpr = kg.get_node_data(edge[1]).get("expression", {}).get("mean", 0)
    edgeType = kg.kg.edges[edge].get("type", "-")
    
    print(edge, edgeType, scoring_gene_gene_expression[edgeType](inExpr, outExpr), inExpr, outExpr)

('NFE2L2', 'CCL2') represses 0.0951349990717979 0.8912855593505586 0.7961505602787607
('CCL13', 'CCL2') interacts 0.6625857304149431 0.8322367193749737 0.7961505602787607
('JUN', 'CCL2') interacts 0.678076627194631 0.8516939647159354 0.7961505602787607
('FOSL1', 'CCL2') activates 0.6385272061656778 0.8020181584022331 0.7961505602787607
('LIF', 'CCL2') interacts 0.9927254232835401 1.2469066440598264 0.7961505602787607
('TNF', 'CCL2') interacts 0.7117152499574244 0.8939455493295483 0.7961505602787607
('TNFSF11', 'CCL2') activates 0.4971813210978469 0.6244815313874376 0.7961505602787607
('CSF2', 'CCL2') interacts 0.0 0 0.7961505602787607
('CSF1', 'CCL2') interacts 0.5629802912872744 0.7071279219977625 0.7961505602787607
('JAK2', 'CCL2') activates 0.7929084270651282 0.9959277385768625 0.7961505602787607
('OLR1', 'CCL2') activates 0.6767790033398968 0.8500640922778882 0.7961505602787607
('FOS', 'CCL2') interacts 0.7147633967622257 0.8977741553205232 0.7961505602787607
('HMOX1', 'CCL2') repr